# 03 – SQL-style EDA using SQLite
Loads the cleaned data into an in-memory SQLite database and runs SQL queries for EDA.

In [ ]:
import pandas as pd
import sqlite3

df = pd.read_csv('../data/economic_indicators_clean.csv')
conn = sqlite3.connect(':memory:')
df.to_sql('econ', conn, index=False, if_exists='replace')
print('Table econ created with', len(df), 'rows')

In [ ]:
# Top 5 country-years by GDP
q1 = '''
SELECT country, year, gdp
FROM econ
ORDER BY gdp DESC
LIMIT 5;
'''
pd.read_sql(q1, conn)

In [ ]:
# Average inflation per country
q2 = '''
SELECT country, AVG(inflation) AS avg_inflation
FROM econ
GROUP BY country
ORDER BY avg_inflation DESC;
'''
pd.read_sql(q2, conn)

In [ ]:
# Worst unemployment year per country
q3 = '''
SELECT country, year, unemployment
FROM econ
WHERE unemployment = (
  SELECT MAX(unemployment) FROM econ e2 WHERE e2.country = econ.country
);
'''
pd.read_sql(q3, conn)

In [ ]:
# Recession rate by country
q4 = '''
SELECT country,
       SUM(recession) * 1.0 / COUNT(*) AS recession_rate
FROM econ
GROUP BY country
ORDER BY recession_rate DESC;
'''
pd.read_sql(q4, conn)